In [1]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os.path

In [3]:
base = pd.read_csv("PETR4.SA.csv")

In [5]:
base = base.dropna()
base_treinamento = base.iloc[:,1:2].values
base_valor_maximo = base.iloc[:,2:3].values
normalizador = MinMaxScaler(feature_range=(0,1))
base_treinamento_normalizada = normalizador.fit_transform(base_treinamento)
base_valor_normalizada = normalizador.fit_transform(base_valor_maximo)


In [15]:
previsores = []
classe1 = []
classe2 = []

for i in range(7,base_treinamento_normalizada.shape[0]):
    previsores.append(base_treinamento_normalizada[i-7:i,0])
    classe1.append(base_treinamento_normalizada[i,0])
    classe2.append(base_valor_normalizada[i,0])
previsores,classe1, classe2 = np.array(previsores),np.array(classe1),np.array(classe2)
previsores = np.reshape(previsores,(previsores.shape[0],previsores.shape[1],1))
classe = np.column_stack((classe1,classe2))


In [20]:
def redeNeuralRecorrenteMultiplasSaidas(prev,cl):
    regressor = Sequential()
    regressor.add(LSTM(units=100 , return_sequences=True, input_shape = (prev.shape[1],1)))
    regressor.add(Dropout(0.3))
    regressor.add(LSTM(units=50, return_sequences=True))
    regressor.add(Dropout(0.3))
    regressor.add(LSTM(units=50, return_sequences=True))
    regressor.add(Dropout(0.3))
    regressor.add(LSTM(units=50))
    regressor.add(Dropout(0.3))
    regressor.add(Dense(units=2,activation='linear')) #retorna valores entre 0-1
    if os.path.isfile('pesosRedeNeuralRecorrente.h5'):
        regressor.load_weights('pesosRedeNeuralRecorrente.h5') 
    regressor.compile(optimizer = 'adam', loss ='mean_squared_error', metrics = ['mean_absolute_error'])
    eas = EarlyStopping(monitor='loss',min_delta=1e-10,patience=5,verbose=1)#Se passarem 10 épocas, e não houver uma mudança mínima de 2e-05, ele para.
    rlc = ReduceLROnPlateau(monitor = 'mean_absolute_error', factor = 0.1,patience=2,verbose=1)#Reduz learning rate em 0.1, se não houver mudanças na loss function
    mcp = ModelCheckpoint(filepath='pesosRedeNeuralRecorrente.h5',monitor='mean_absolute_error',save_best_only=True)
    regressor.fit(prev,cl,epochs = 100, batch_size = 32,callbacks=[eas,rlc,mcp])
    return regressor

In [22]:
kfold = KFold(n_splits=15)
for train, test in kfold.split(previsores, classe):
  regressor = redeNeuralRecorrenteMultiplasSaidas(previsores[train], classe[train])

Epoch 1/100
155/155 [==============================] - 8s 19ms/step - loss: 0.0016 - mean_absolute_error: 0.0271
Epoch 2/100
155/155 [==============================] - 3s 19ms/step - loss: 0.0012 - mean_absolute_error: 0.0235
Epoch 3/100
155/155 [==============================] - 3s 19ms/step - loss: 0.0014 - mean_absolute_error: 0.0232
Epoch 4/100
155/155 [==============================] - 3s 19ms/step - loss: 0.0017 - mean_absolute_error: 0.0232
Epoch 5/100
155/155 [==============================] - 3s 19ms/step - loss: 9.8409e-04 - mean_absolute_error: 0.0217
Epoch 6/100
155/155 [==============================] - 3s 19ms/step - loss: 0.0011 - mean_absolute_error: 0.0204
Epoch 7/100
155/155 [==============================] - 3s 19ms/step - loss: 9.4347e-04 - mean_absolute_error: 0.0213
Epoch 8/100
155/155 [==============================] - 3s 19ms/step - loss: 7.3687e-04 - mean_absolute_error: 0.0188
Epoch 9/100
155/155 [==============================] - 3s 19ms/step - loss: 9.1422e-

In [34]:
x_teste = [5.338000,	5.363000,5.325000,5.313000,5.200000,5.163000,5.163000] #5.225000
x_teste = np.array(x_teste)
x_teste = x_teste.reshape(-1,1)
x_teste = normalizador.transform(x_teste)
x_teste = x_teste.reshape(1, 7, 1)
previsao = regressor.predict(x_teste)
previsao = normalizador.inverse_transform(previsao)
previsao

array([[5.566598 , 5.6125197]], dtype=float32)